In [64]:
import numpy as np
import pandas as pd
import os
import sklearn
import rpy2

In [65]:
# Load data 
antonPath = False
alexPath = True 
if alexPath == True :
    wd = "C:\\Users\\alexa\\Desktop\\FixedIncomeStrat"
else:
    wd = "/Users/antonaleynikov/Desktop/Fixed Income Sentiment strat"
os.chdir(wd)
os.getcwd()

'c:\\Users\\alexa\\Desktop\\FixedIncomeStrat'

In [66]:
for path, folders, files in os.walk(wd):
    print("folders:", folders)
    print("files:", files)
    print("path:", path)

folders: ['.git', 'Data']
files: ['.DS_Store', 'jupyter_notebook_config.py', 'README.md', 'US_Bonds_Butterflies_Strat.ipynb']
path: C:\Users\alexa\Desktop\FixedIncomeStrat
folders: ['hooks', 'info', 'logs', 'objects', 'refs']
files: ['config', 'description', 'FETCH_HEAD', 'HEAD', 'index', 'ORIG_HEAD', 'packed-refs']
path: C:\Users\alexa\Desktop\FixedIncomeStrat\.git
folders: []
files: ['applypatch-msg.sample', 'commit-msg.sample', 'fsmonitor-watchman.sample', 'post-update.sample', 'pre-applypatch.sample', 'pre-commit.sample', 'pre-push.sample', 'pre-rebase.sample', 'pre-receive.sample', 'prepare-commit-msg.sample', 'update.sample']
path: C:\Users\alexa\Desktop\FixedIncomeStrat\.git\hooks
folders: []
files: ['exclude']
path: C:\Users\alexa\Desktop\FixedIncomeStrat\.git\info
folders: ['refs']
files: ['HEAD']
path: C:\Users\alexa\Desktop\FixedIncomeStrat\.git\logs
folders: ['heads', 'remotes']
files: []
path: C:\Users\alexa\Desktop\FixedIncomeStrat\.git\logs\refs
folders: []
files: ['main

In [67]:
data_path = os.path.join(wd, "Data")
if alexPath == True :
    datafiles_names = os.listdir(wd + "\\Data")
else :
    datafiles_names = os.listdir(wd + "/Data")
    

filenames = [filename.split(".") for filename in datafiles_names[2:]]


In [68]:
for file in filenames:
    file_name = file[0]
    file_format = file[1]
    file_path = os.path.join(data_path , ".".join(file))
    df = pd.read_excel(file_path, index_col = "Dates")
    if file_format == "xlsx":
        df = pd.read_excel(file_path, index_col = "Dates")
    globals()[file_name] = df 
locals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import numpy as np\nimport pandas as pd\nimport os',
  'for file in filenames:\n    file_name = file[0]\n    file_format = file[1]\n    file_path = os.path.join(data_path , ".".join(file))\n    df = pd.read_excel(file_path, index_col = "Dates")\n    if file_format == "xlsx":\n        df = pd.read_excel(file_path, index_col = "Dates")\n    globals()[file_name] = df \nlocals()',
  'data_path = os.path.join(wd, "Data")\nif alexPath == True :\n    datafiles_names = os.listdir(wd + "\\\\Data")\nelse :\n    datafiles_names = os.listdir(wd + "/Data")\n    \n\nfilenames = [filename.split(".") for filename in datafiles_names[2:]]',
  'import numpy as np\nimport pandas as pd\nimport os',
  '# Load data \nantonPath = Fals

In [69]:
Yields.head()

,DGS1MO,DGS3MO,DGS6MO,DGS1,DGS2,DGS3,DGS5,DGS7,DGS10,DGS20,DGS30
Dates,,,,,,,,,,,
2003-01-02,1.18,1.22,1.25,1.42,1.80,2.22,3.05,3.62,4.07,5.05,5.18
2003-02-03,1.16,1.18,1.21,1.34,1.72,2.17,3.05,3.58,4.01,4.93,5.01
2003-03-03,1.20,1.20,1.21,1.26,1.52,1.89,2.66,3.21,3.68,4.69,4.92
2003-04-01,1.17,1.12,1.10,1.16,1.50,1.92,2.78,3.35,3.84,4.85,5.06
2003-05-01,1.09,1.10,1.13,1.21,1.50,1.93,2.82,3.37,3.88,4.80,5.01


In [70]:
Exogen["SPX_EP_YIELD_SPREAD"] = Exogen["SPX_PE_RATIO"].apply(lambda x: 1 / x) - (Yields[1:]["DGS20"]/100).values
del Exogen["SPX_PE_RATIO"]
Exogen

,unk,USYC3M10 Index,SPX_GROSS_AGGTE_DVD_YLD,SPX_EP_YIELD_SPREAD
Dates,,,,
2003-02-28,-0.45,249.704,1.90,0.001084
2003-03-31,-0.18,269.405,1.88,0.019077
2003-04-30,-0.65,272.061,1.75,0.011914
2003-05-31,0.01,226.873,1.67,0.005934
2003-06-30,0.12,266.133,1.66,0.008927
...,...,...,...,...
2022-11-30,-0.55,-77.371,1.64,0.008100
2022-12-31,-0.47,-53.018,1.76,0.016390
2023-01-31,0.41,-118.137,1.66,0.011095


In [71]:
from sklearn.linear_model import LinearRegression as lm 


def get_lam(yields, lam_seq):
    "get lambda given known yields and a searching grid"
    if 0 in lam_seq:
        print("Remove zero from search grid")
    maturities = np.array([1/12, 1/4,1/2 ,1 , 2, 3, 5, 7, 10, 20, 30])
    r2 = pd.DataFrame(0, index=np.arange(yields.shape[0]), columns=lam_seq)
    for j in range(len(lam_seq)):
        lam = lam_seq[j]
        
        # Initialise regressors based on lambda 
        regs = pd.DataFrame()
        regs["level_reg"] = [1 for t in maturities]
        regs["slope_reg"] = [(1 - np.exp(-lam * t))/(lam * t) for t in maturities]
        regs["curv_reg"] = [(1 - np.exp(-lam * t))/(lam * t) - np.exp(-lam*t) for t in maturities]
        
        # Fit yields to regressors
        factors = pd.DataFrame({"level": "", "slope": "", "curvature": ""},index = yields.index)
        for i in range(yields.shape[0]):
            X = regs.values
            y = yields.iloc[i].values
            reg = lm().fit(X, y)
            level, slope ,curv = reg.intercept_, reg.coef_[1], reg.coef_[2]
            rsq = reg.score(X,y)
            factors.iloc[i] = [level, slope, curv]
            r2.iloc[i,j] = rsq
    lam_max = r2.mean().idxmax()
    return lam_max
        
# get_lam(yields, np.linspace(0.1,1,10))

def fit_factors(yields, lam):
    factors = pd.DataFrame({"level": "", "slope": "", "curvature": ""},index = yields.index)
    
    # Initialise regressors based on lambda
    maturities = np.array([1/12, 1/4,1/2 ,1 , 2, 3, 5, 7, 10, 20, 30])
    regs = pd.DataFrame()
    regs["level_reg"] = [1 for t in maturities]
    regs["slope_reg"] = [(1 - np.exp(-lam * t))/(lam * t) for t in maturities]
    regs["curv_reg"] = [(1 - np.exp(-lam * t))/(lam * t) - np.exp(-lam*t) for t in maturities]
    
    for row in range(factors.shape[0]):
        reg = lm().fit(regs.values, yields.iloc[row].values)
        level = reg.intercept_
        slope , curv = reg.coef_[1], reg.coef_[2]
        factors.iloc[row] = [level, slope, curv]
    return factors

# lam = get_lam(yields, np.linspace(0.1,1,10))
# factors = fit_factors(yields, lam)

In [72]:
lam = get_lam(Yields, np.linspace(0.1,1,10))
factors = fit_factors(Yields, lam)
dfact = factors.diff()[1:]

C:\Users\alexa\AppData\Local\Temp\ipykernel_27384\3118424711.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9967262008321095' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  r2.iloc[i,j] = rsq


C:\Users\alexa\AppData\Local\Temp\ipykernel_27384\3118424711.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.994411060475153' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  r2.iloc[i,j] = rsq
C:\Users\alexa\AppData\Local\Temp\ipykernel_27384\3118424711.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.995172030220602' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  r2.iloc[i,j] = rsq
C:\Users\alexa\AppData\Local\Temp\ipykernel_27384\3118424711.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.997336634130186' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  r2.iloc[i,j] = rsq
C:\Users\alexa\AppData\Local\Temp\ipykernel_27384\3118424711.

In [73]:
# TEST
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector
from rpy2.robjects import pandas2ri
utils = importr("utils")
# utils.chooseCRANmirror(ind=1)
# utils.install_packages('rugarch')
# utils.install_packages('forecast')
forecast = importr('forecast')
rugarch = importr('rugarch')
base = importr('base')

# X is a Panda Series that contains the returns I want to fit the model
returns = FloatVector(dfact["slope"].values)   
with (ro.default_converter + pandas2ri.converter).context():
     exo = ro.conversion.get_conversion().py2rpy(Exogen)

ro.r("""  
     f <- function (series, exogen) {
     spec = ugarchspec(
     variance.model=list(garchOrder=c(1,1)),
     mean.model=list(armaOrder=auto.arima(series)$arma[1:2], external.regressors = exogen)
     ) 

     fit = tryCatch(
     ugarchfit(
     spec, series, solver = 'hybrid'
     ), error=function(e) e, warning=function(w) w
     )  
     fore = ugarchforecast(fit, n.ahead=1)
     ind = fore@forecast$seriesFor
     ind
     }""")

fcast_armax = ro.globalenv['f']
fore = fcast_armax(returns, base.as_matrix(exo))[0]

fore

0.14164051365136623

In [74]:
s_len = dfact.shape[0] 
win = 160
h = 1
arr = []
for t in range (s_len - win - h):
    end = t + win

    series = FloatVector(dfact["slope"][t:end].values)   
    with (ro.default_converter + pandas2ri.converter).context():
        exo = ro.conversion.get_conversion().py2rpy(Exogen[t:end])
    
    fore = fcast_armax(series, base.as_matrix(exo))[0]
    arr.append(fore)